In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import csv
from statistics import median

import surprise
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [4]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
user_and_item_data71mb = []

f = gzip.open("user_and_item_data71mb.json.gz")
for l in f:
    d = eval(l)
    user_and_item_data71mb.append(d)
    if len(user_and_item_data71mb) >= 50000:
        break
        
f.close()

In [6]:
review_data_6mb = []

f = gzip.open("review_data_6mb.json.gz")
for l in f:
    d = eval(l)
    review_data_6mb.append(d)
    if len(review_data_6mb) >= 50000:
        break
        
f.close()

In [7]:
review_data_6mb[4]

{'user_id': 'maplemage',
 'user_url': 'http://steamcommunity.com/id/maplemage',
 'reviews': [{'funny': '3 people found this review funny',
   'posted': 'Posted April 15, 2014.',
   'last_edited': '',
   'item_id': '211420',
   'helpful': '35 of 43 people (81%) found this review helpful',
   'recommend': True,
   'review': 'Git gud'},
  {'funny': '1 person found this review funny',
   'posted': 'Posted December 23, 2013.',
   'last_edited': '',
   'item_id': '211820',
   'helpful': '12 of 16 people (75%) found this review helpful',
   'recommend': True,
   'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
  {'funny': '2 people found this review funny',
   'posted': 'Posted March 14, 2014.',
   'last_edited': '',
   'item_id': '730',
   'helpful': '5 of 5 people (100%) found this review helpful',
   'recommend': True,
   'review': 'Hold shift to win, Hold CTRL to lose.'},
  {'funny': '',
   'posted': 'Posted July 

In [8]:
user_and_item_data71mb[5]

{'user_id': 'MinxIsBetterThanPotatoes',
 'items_count': 371,
 'steam_id': '76561198004744620',
 'user_url': 'http://steamcommunity.com/id/MinxIsBetterThanPotatoes',
 'items': [{'item_id': '50',
   'item_name': 'Half-Life: Opposing Force',
   'playtime_forever': 256,
   'playtime_2weeks': 0},
  {'item_id': '240',
   'item_name': 'Counter-Strike: Source',
   'playtime_forever': 167,
   'playtime_2weeks': 0},
  {'item_id': '320',
   'item_name': 'Half-Life 2: Deathmatch',
   'playtime_forever': 3674,
   'playtime_2weeks': 0},
  {'item_id': '4000',
   'item_name': "Garry's Mod",
   'playtime_forever': 358,
   'playtime_2weeks': 0},
  {'item_id': '6570',
   'item_name': 'Onimusha 3: Demon Siege',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '220',
   'item_name': 'Half-Life 2',
   'playtime_forever': 1299,
   'playtime_2weeks': 0},
  {'item_id': '340',
   'item_name': 'Half-Life 2: Lost Coast',
   'playtime_forever': 29,
   'playtime_2weeks': 0},
  {'item_id': '360',
  

In [9]:
len(user_and_item_data71mb)

50000

In [10]:
len(review_data_6mb)

25799

In [11]:
item_metadata = []

f = gzip.open("item_metadata.json.gz")
for l in f:
    d = eval(l)
    item_metadata.append(d)
    if len(item_metadata) >= 20000:
        break
        
f.close()

In [12]:
item_metadata[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [13]:
def merge_datasets_with_matching(review_data, user_and_item_data):
    """
    Merge datasets by matching user IDs with extensive debugging
    
    :param review_data: List of user review data
    :param user_and_item_data: List of user item and playtime data
    :return: List of merged data entries
    """
    # Verify input dataset lengths
    print(f"Original Review Data Length: {len(review_data)}")
    print(f"Original Items Data Length: {len(user_and_item_data)}")
    
    # Create lookup dictionaries for efficient matching
    review_dict = {entry['user_id']: entry for entry in review_data}
    items_dict = {entry['user_id']: entry for entry in user_and_item_data}
    
    # Detailed user ID analysis
    review_user_ids = set(review_dict.keys())
    items_user_ids = set(items_dict.keys())
    
    print(f"Unique User IDs in Review Data: {len(review_user_ids)}")
    print(f"Unique User IDs in Items Data: {len(items_user_ids)}")
    
    # Find common user IDs
    common_user_ids = review_user_ids & items_user_ids
    
    # More detailed tracking
    unmatched_review_users = review_user_ids - items_user_ids
    unmatched_items_users = items_user_ids - review_user_ids
    
    print(f"Common User IDs: {len(common_user_ids)}")
    print(f"Unmatched Users in Review Data: {len(unmatched_review_users)}")
    print(f"Unmatched Users in Items Data: {len(unmatched_items_users)}")
    
    # Sample of unmatched user IDs for investigation
    print("\nSample of Unmatched Review User IDs:")
    print(list(unmatched_review_users)[:10])
    print("\nSample of Unmatched Items User IDs:")
    print(list(unmatched_items_users)[:10])
    
    # Store merged data
    merged_data = []
    
    # Tracking variables
    total_reviews_processed = 0
    total_merged_entries = 0
    
    # Process only users present in both datasets
    for user_id in common_user_ids:
        user_reviews = review_dict[user_id]['reviews']
        user_items = items_dict[user_id]['items']
        
        total_reviews_processed += len(user_reviews)
        
        # Match reviews with items
        for review in user_reviews:
            # Find matching items
            matching_items = [
                item for item in user_items 
                if str(item['item_id']) == str(review['item_id'])
            ]
            
            # Create merged entries for matching items
            for item in matching_items:
                merged_entry = {
                    'user_id': user_id,
                    'item_id': review['item_id'],
                    'review_text': review.get('review', ''),
                    'recommend': review.get('recommend'),
                    'playtime_forever': item.get('playtime_forever', 0),
                    'playtime_2weeks': item.get('playtime_2weeks', 0)
                }
                merged_data.append(merged_entry)
                total_merged_entries += 1
    
    # Comprehensive summary
    print("\nFinal Matching Summary:")
    print(f"Total Users in Review Data: {len(review_dict)}")
    print(f"Total Users in Items Data: {len(items_dict)}")
    print(f"Common Users: {len(common_user_ids)}")
    print(f"Total Reviews Processed: {total_reviews_processed}")
    print(f"Total Merged Entries: {total_merged_entries}")
    
    return merged_data

In [14]:
merged_data = merge_datasets_with_matching(review_data_6mb,user_and_item_data71mb)

Original Review Data Length: 25799
Original Items Data Length: 50000
Unique User IDs in Review Data: 25485
Unique User IDs in Items Data: 49320
Common User IDs: 19248
Unmatched Users in Review Data: 6237
Unmatched Users in Items Data: 30072

Sample of Unmatched Review User IDs:
['76561198048691489', 'sparid', 'Big_Kill', 'me2118', 'icedcoconut', '76561198132273399', '76561198103382674', '76561198069821382', '76561198098845763', '76561198103845573']

Sample of Unmatched Items User IDs:
['Robert5292', 'llamasrawesum', '76561198039969558', '76561198099310155', '76561198021402409', '76561198074073622', 'tearanosinn', 'coleridge49', 'iknowdude00', 'wonglepong']

Final Matching Summary:
Total Users in Review Data: 25485
Total Users in Items Data: 49320
Common Users: 19248
Total Reviews Processed: 46257
Total Merged Entries: 35600


In [15]:
merged_data[8]

{'user_id': '76561198088213120',
 'item_id': '4000',
 'review_text': "Awwwhhhh m8! Met great friends on Garry's Mod and have spent a lot of my useless life on it :D",
 'recommend': True,
 'playtime_forever': 38753,
 'playtime_2weeks': 0}

In [16]:
unique_user_ids = len(set(entry['user_id'] for entry in merged_data))
print(f"Number of unique user IDs: {unique_user_ids}")

# To get number of unique item IDs
unique_item_ids = len(set(entry['item_id'] for entry in merged_data))
print(f"Number of unique item IDs: {unique_item_ids}")

# If you want to see both at once
print("Unique user IDs:", len(set(entry['user_id'] for entry in merged_data)))
print("Unique item IDs:", len(set(entry['item_id'] for entry in merged_data)))

Number of unique user IDs: 15379
Number of unique item IDs: 2911
Unique user IDs: 15379
Unique item IDs: 2911


In [17]:
# Count recommendations
recommendations = [entry['recommend'] for entry in merged_data]

# Using numpy for a quick count
import numpy as np
unique, counts = np.unique(recommendations, return_counts=True)
print("Recommendation Distribution:")
print(dict(zip(unique, counts)))

# Alternatively, using a simple count method
true_count = recommendations.count(True)
false_count = recommendations.count(False)

print("\nRecommend True Count:", true_count)
print("Recommend False Count:", false_count)
print("Percentage of Recommendations:", true_count / len(recommendations) * 100, "%")

Recommendation Distribution:
{False: 4178, True: 31422}

Recommend True Count: 31422
Recommend False Count: 4178
Percentage of Recommendations: 88.26404494382022 %


In [18]:
import random

# Separate recommended and non-recommended entries
recommend_true = [entry for entry in merged_data if entry['recommend'] == True]
recommend_false = [entry for entry in merged_data if entry['recommend'] == False]

# Calculate target number of True recommendations (about 66%)
total_desired = len(recommend_false) * 1  # keep all False entries, add same number of True

# Randomly sample from True recommendations
balanced_recommend_true = random.sample(recommend_true, total_desired)

# Recombine the datasets
balanced_merged_data = recommend_false + balanced_recommend_true

# Shuffle the balanced dataset
random.shuffle(balanced_merged_data)

# Verify new distribution
recommendations = [entry['recommend'] for entry in balanced_merged_data]
unique, counts = np.unique(recommendations, return_counts=True)
print("New Recommendation Distribution:")
print(dict(zip(unique, counts)))

New Recommendation Distribution:
{False: 4178, True: 4178}


In [19]:
balanced_merged_data[0]

{'user_id': 'Fried_Bacon',
 'item_id': '266840',
 'review_text': 'Better off getting the original game.',
 'recommend': False,
 'playtime_forever': 161,
 'playtime_2weeks': 0}

In [20]:
len(balanced_merged_data)

8356

In [52]:
X = [[entry['playtime_forever'] + entry['playtime_forever'],len(entry['review_text']),  # number of question marks
      1]  # bias term (constant feature)
     for entry in balanced_merged_data]

y = [entry['recommend'] for entry in balanced_merged_data]

In [22]:
len(X)

8356

### setting len(recommend_false) * 1 and no weght_class balanced gives slighly better performance

In [56]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

# Train logistic regression
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

# Evaluate
#print("Model Score:", mod.score(X_test, y_test))
#print("Coefficients:", mod.coef_)
predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.5921493537577789


In [57]:
# Random Baseline Classifier
baseline = DummyClassifier(strategy='stratified', random_state=42)
baseline.fit(X_train, y_train)
baseline_pred = baseline.predict(X_test)

print("Baseline (Random) Accuracy:", accuracy_score(y_test, baseline_pred))
print("\nBaseline Classification Report:")
print(classification_report(y_test, baseline_pred))

print("\nLogistic Regression Predictions:")
print(classification_report(y_test, mod.predict(X_test)))

Baseline (Random) Accuracy: 0.5093346098611776

Baseline Classification Report:
              precision    recall  f1-score   support

       False       0.50      0.51      0.50      1029
        True       0.52      0.51      0.51      1060

    accuracy                           0.51      2089
   macro avg       0.51      0.51      0.51      2089
weighted avg       0.51      0.51      0.51      2089


Logistic Regression Predictions:
              precision    recall  f1-score   support

       False       0.57      0.73      0.64      1029
        True       0.64      0.46      0.53      1060

    accuracy                           0.59      2089
   macro avg       0.60      0.59      0.59      2089
weighted avg       0.60      0.59      0.58      2089



In [58]:
TP = [a and b for (a,b) in zip(predictions,y)]
TN = [not a and not b for (a,b) in zip(predictions,y)]
FP = [a and not b for (a,b) in zip(predictions,y)]
FN = [not a and b for (a,b) in zip(predictions,y)]

In [59]:
TP = sum(TP)
TN = sum(TN)
FP = sum(FP)
FN = sum(FN)

In [60]:
BER = 0.5 * (FP / (TN + FP) + FN / (FN + TP))


In [61]:
BER

0.5058884193350262

### n-grams

In [62]:
sentence = "Dark red color, light beige foam"
unigrams = sentence.split()
bigrams = list(zip(unigrams[:-1], unigrams[1:]))
trigrams = list(zip(unigrams[:-2], unigrams[1:-1], unigrams[2:]))

In [63]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))
for d in balanced_merged_data:
    review_text = d.get('review_text', '').lower()  # Default to empty string if key is missing

    # Remove punctuation
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter stopwords
    ws = [w for w in r.split() if w not in stop_words]
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
    ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
    ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
    for w in ws + ws2 + ws3 + ws4 + ws5:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [64]:
len(counts)

793022

In [65]:
words = [x[1] for x in counts[:2000]]

In [66]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [67]:
def feature(datum):
    # Initialize feature vector with zeros
    feat = [0] * len(words)

    # Safely access and preprocess 'review_text'
    review_text = datum.get('review_text', '').lower()
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter out stopwords
    ws = [w for w in r.split() if w not in stop_words]

    # Generate n-grams (1-grams to 5-grams)
    n_grams = []
    for n in range(1, 6):  # 1-grams to 5-grams
        n_grams += [' '.join(ws[i:i + n]) for i in range(len(ws) - n + 1)]

    # Populate feature vector based on n-grams in 'words'
    for w in n_grams:
        if w in words:
            feat[wordId[w]] += 1

    # Add offset for bias term
    feat.append(1)
    return feat

In [68]:
X = [feature(d) for d in balanced_merged_data]
y = [d['recommend'] for d in balanced_merged_data]

In [69]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

In [70]:
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.7865007180469124


### tf - idf + classifier 

### maybe jacard or cos sim

### data processing:
### grab t

In [71]:
# Count unique users
unique_users = len(set(entry['user_id'] for entry in balanced_merged_data))
print("Number of Unique Users:", unique_users)

# Count unique items
unique_items = len(set(entry['item_id'] for entry in balanced_merged_data))
print("Number of Unique Items:", unique_items)

# Count all items 


Number of Unique Users: 5938
Number of Unique Items: 1676


In [72]:
balanced_merged_data[0]

{'user_id': 'Fried_Bacon',
 'item_id': '266840',
 'review_text': 'Better off getting the original game.',
 'recommend': False,
 'playtime_forever': 161,
 'playtime_2weeks': 0}

In [73]:
balanced_merged_data_10 = [
    {**entry, 'recommend': 1 if entry['recommend'] else 0} 
    for entry in balanced_merged_data
]

In [88]:
recommendTrain = balanced_merged_data_10[:5000]
recommendValid = balanced_merged_data_10[5000:]
recommendPerUser = defaultdict(list)
recommendPerItem = defaultdict(list)
for entry in recommendTrain:
    u = entry['user_id']
    i = entry['item_id']
    r = entry['recommend']  # Now already 0/1
    
    recommendPerUser[u].append((i,r))
    recommendPerItem[i].append((u,r))

In [89]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [90]:
trainRatings = [r['recommend'] for r in recommendValid]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)

In [91]:
globalAverage

0.499404052443385

In [92]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in recommendTrain:
    user,item = d['user_id'], d['item_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)
    ratingDict[(user,item)] = d['recommend']

In [93]:
userAverages = {}
itemAverages = {}
ratingMean = []

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)
    
for d in recommendTrain:
    ratingMean.append(d['recommend'])
    
ratingMean = sum(ratingMean) / len(ratingMean)

In [94]:
def MSE(y, ypred):
    diffs = [(a-b)**2 for (a,b) in zip(y,ypred)]
    return sum(diffs) / len(diffs)

In [95]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['item_id']
        if i2 == item: continue
        ratings.append(d['recommend'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [96]:
alwaysPredictMean = [ratingMean for d in recommendValid]

In [97]:
simPredictions = [predictRating(d['user_id'], d['item_id']) for d in recommendValid]

In [98]:
labels = [d['recommend'] for d in recommendValid]

In [99]:
MSE(alwaysPredictMean, labels)

0.2500006367580453

In [100]:
MSE(simPredictions, labels)

0.2550808602848265